In [1]:
import pandas as pd
import re
from sqlalchemy import (MetaData, Table, Column, Integer, Numeric, String,
                        DateTime, ForeignKey, create_engine, select)
import mysql.connector

In [2]:
# Static Database values
db_username = 'root'
db_password = 'Passw0rd'
db_ip = 'localhost'
db_name = 'Bank'

db_u = 'avn_monitor'
db_p = 'Pass@123'
db_host = '192.168.1.30'
db = 'Req_data'
# Connection for MySQL server

engine = create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                       format(db_username, db_password,
                              db_ip, db_name), pool_pre_ping=True, pool_recycle=3600)

# MSSQL Connection
msengine = create_engine('mssql+pymssql://{0}:{1}@{2}/{3}'.
                         format(db_u, db_p,
                                db_host, db), pool_pre_ping=True, pool_recycle=3600)

In [6]:
metadata = MetaData()
table = Table('BankData', metadata, autoload=True, autoload_with=engine)

In [11]:
table.columns.bank_

In [12]:
s = select([table]).limit(1)
print(engine.execute(s).fetchall())

[(1, 'STATE BANK OF INDIA', 'SBIN0018895', 'MARLAPADU', 'TIRUVURU ROAD,MARLAPADU,DIST KHAMMAM, TELANGANA-507164', '9100050724', 'MARLAPADU', 'KHAMMAM', 'TELANGANA')]


In [32]:
df = pd.read_sql_table(table_name='BankData', con=engine, index_col=False)

In [33]:
df.head()

,bank_id,bank_name,bank_ifsc_code,bank_branch,bank_address,bank_contact,bank_city,bank_district,bank_state
0,1,STATE BANK OF INDIA,SBIN0018895,MARLAPADU,"TIRUVURU ROAD,MARLAPADU,DIST KHAMMAM, TELANGAN...",9100050724,MARLAPADU,KHAMMAM,TELANGANA
1,2,STATE BANK OF INDIA,SBIN0016892,RAIPUR RANI,"TRILOKPUR CHOWK,RAIPUR RANI,DISTT.PANCHKULA.HA...",9666146461,PANCHKULA,PANCHKULA,HARYANA
2,3,STATE BANK OF INDIA,SBIN0016932,ROWRIAH,"ROWRIAH TINALI,P.O ROWRIAH,JORHAT,DISTT.JORHAT...",9401834053,JORHAT,JORHAT,ASSAM
3,4,STATE BANK OF INDIA,SBIN0017032,KODUVAYUR,"PUDUNAGARAM ROAD,KODUVAYUR P.O,KODUVAYUR,DISTT...",9447010238,PALAKKAD,PALAKKAD,KERALA
4,5,STATE BANK OF INDIA,SBIN0017067,NOORUDDINPUR ZILLA AMETHI,"NOORUDDINPUR,VILL.AND POST KARIHAIYA BAZAR,TEH...",7897007227,AMETHI,AMETHI,UTTAR PRADESH


In [34]:
df.drop(['bank_id'],axis=1, inplace=True)

In [36]:
df1 = df.head(1)

In [39]:
df1.to_excel("test.xls")

In [25]:
a = "-- @ Title\t: ccav$assign_payouts\r\n-- @ Version : 1.0\r\n-- @ Author : Rakesh\r\n-- @ Company : Avenues (I) Pvt. Limited.\r\n-- @ Description : Assigning Payouts\r\n-- @ Created : 12-Dec-2011\r\n-- @ Modified : 15-SEP-2016 Supriya Ghude - Reg_id - 104847 removed from the list\r\n-- @ Modified : 17-SEP-2018 Supriya Ghude - Added code for - [td_charge_TDR]\r\n-- @ Modified : 02-OCT-2018 Supriya Ghude - Added code for - [stl_include_debit]\r\n-- @ Modified : 06-SEP-2018 Kalyani Pandey - Added td_trans_type = 'MSGCHRG' with 'MISC'\r\n-- @ Modified : 24-April-2019 Dinesh - Changed condition for @batch_cutoff_date date format in WHERE clause.\r\n-- @ Modified : 25-APR-2019 Supriya Ghude - Code changes for - td_charge_TDR\r\n-- @ Modified : 17-MAY-2019 Kalyani Pandey - Added 'CC-ICICI','CC-HDFC','CC-KMB' nodal with 'CCAvenue' in order_stlmt_to\r\n-- @ Modified : 22-AUG-2019 Kalyani Pandey - Added code for checking reseller with setting_rsl_consolidated_stlmt = 'Y' for payout\r\n--\t\t\t\t\t\t\t\t\t\t\t- Added code for consolidate payout reseller wise for @activity = 'CLOSED'\r\n/*\r\n\tSElect * from ccav_payout_batch where batch_id = 37\r\n\t\r\n\tUPDATE CCAV_payout_detail_temp \r\n\t\tSET td_status = 'AWTD',\r\n\t\t\ttd_batch_id = NULL\r\n\tWHERE td_batch_id = 42\r\n\t\r\n\tDelete from CCAV_payout_detail_temp where td_id = 322\r\n\r\n\tSELECT * from CCAV_payout_detail_temp \r\n\twhere reg_id = 2660\r\n\t\r\n\tSelect * from ccav_payout_detail_temp where td_batch_id = 42\r\n\texec ccav$assign_payouts '2660,',42,'EUR','DVP','11-02-2013','ASSIGNED'\r\n\t\r\n\tSelect * from exception_log order by event_occured desc\r\n\t\r\nUPDATE\tccav_merchant_gateway_setting \r\n\tSET setting_charge_tdr = 'N'\r\nWhere reg_id = 81724\r\n\r\nSelect setting_charge_tdr from ccav_merchant_gateway_setting \r\nWhere reg_id = 81724\r\n\r\nDeclare @p_iserror\t\t\t\tVARCHAR(2)  ,@p_error_desc\t\t\tVARCHAR(300)\r\nEXEC [ccav$sub_assign_payouts] '196907|,',20719,'INR','NDVP','10-01-2019','ASSIGNED','ICICI',@p_iserror OUT,@p_error_desc OUT\r\nselect @p_iserror ,@p_error_desc \r\n*/\r\nCREATE PROCEDURE [dbo].[ccav$sub_assign_payouts](\r\n\t@reg_ids\t\t\t\tVARCHAR(8000),\r\n\t@batch_id\t\t\t\tINT,\r\n\t@batch_curr_id\t\t\tVARCHAR(3),\r\n\t@batch_merchant_type\tVARCHAR(4),\r\n\t@batch_cutoff_date\t\tVARCHAR(22),\r\n\t@activity\t\t\t\tVARCHAR(10),\r\n\t@p_batch_gtw_id\t\t\tVARCHAR(10),\r\n\t@p_iserror\t\t\t\tVARCHAR(2)  OUTPUT,\r\n\t@p_error_desc\t\t\tVARCHAR(300) OUTPUT\r\n\t)\r\nAS\r\nSET NOCOUNT ON\r\n\r\nBEGIN\r\nBEGIN TRY\r\n\r\nDECLARE @reg_id VARCHAR(100),\r\n\t\t@v_reg_id\tINT,\r\n\t\t@v_sub_acc_id\tVARCHAR(20),\r\n\t\t@td_payout_id INT,\r\n\t\t@v_batch_cutoff_date\tDATETIME,\r\n\t\t@v_rsl_id\t\tINT\r\n\t\t\r\nCREATE TABLE #temp_payout(\r\n\tid\tINT IDENTITY(1,1),\r\n\treg_id INT,\r\n\tpay_to\tVARCHAR(255),\r\n\ttd_type\tVARCHAR(5),\r\n\tpay_amt\tNUMERIC(13,3),\r\n\tbank_transfer CHAR(1),\r\n\trcnt\tINT,\r\n\ttd_stl_curr_id CHAR(3), --record count\r\n\tsub_acc_id\tVARCHAR(20),\r\n\ttd_charge_TDR CHAR(1)\r\n\t)\r\n\r\nSELECT\t@v_rsl_id = batch_rsl_id \r\nFROM\tCCAV_payout_batch\r\nWHERE\tbatch_id = @batch_id\r\n\t\r\nIF @activity = 'ASSIGNED'\r\nBEGIN\r\n\r\nBEGIN TRAN Assigned_payout\r\nWHILE LEN(@reg_ids)>0\r\nBEGIN\r\n\t\r\n\tSET @reg_id = NULL\r\n\tSET @td_payout_id = NULL\r\n\tTRUNCATE TABLE #temp_payout\r\n\r\n\tSET @reg_id = SUBSTRING(@reg_ids,1,CHARINDEX(',',@reg_ids)-1)\r\n\tSET @v_reg_id = SUBSTRING(@reg_id,1,CHARINDEX('|',@reg_id)-1)\r\n\tSET @v_sub_acc_id = SUBSTRING(@reg_id,CHARINDEX('|',@reg_id)+1,LEN(@reg_id))\r\n\tSET @reg_ids = SUBSTRING(@reg_ids,CHARINDEX(',',@reg_ids)+1,LEN(@reg_ids))\r\n\t\r\n\tIF ISNULL(@v_rsl_id,'') <> '' -- Checking whether reg_id belongs to reseller with setting_rsl_consolidated_stlmt = 'Y'\r\n\t\tBEGIN\r\n\t\tIF NOT EXISTS(SELECT 1 FROM CCAV_registration WHERE reg_id = @v_reg_id AND rsl_id = @v_rsl_id)\r\n\t\t\tCONTINUE\r\n\t\tEND\r\n\tELSE \r\n\t\tBEGIN\r\n\t\tIF EXISTS(SELECT 1 FROM CCAV_registration A\r\n\t\t\t\tINNER JOIN ccav_reseller_setting B ON A.rsl_id = B.rsl_id\r\n\t\t\t\tWHERE A.reg_id = @v_reg_id\r\n\t\t\t\t\tAND B.setting_rsl_consolidated_stlmt = 'Y')\r\n\t\t\tCONTINUE\r\n\t\tEND\r\n\r\n\t--IF @batch_id = 492 OR @reg_id = 42004\r\n\t--\tSELECT @v_batch_cutoff_date = batch_cutoff_date from CCAV_payout_batch WHERE batch_id = 492\r\n\r\n\tUPDATE\tA\r\n\t\tSET A.td_batch_id = @batch_id,\r\n\t\t\tA.td_status = 'ASSIGNED',\r\n\t\t\tA.td_stl_curr_id = B.stl_curr_id\r\n\tFROM\tCCAV_payout_detail_temp A, CCAV_stl_bank_detail B --,CCAV_curr_conversion C\r\n--\tWHERE\tA.td_date <= @v_batch_cutoff_date --CONVERT(DATETIME,@batch_cutoff_date,105)\r\n\tWHERE\tA.td_date <= CONVERT(DATETIME,@batch_cutoff_date,105)\r\n\t\tAND A.td_curr_id = @batch_curr_id\r\n\t\tAND A.reg_id = B.reg_id \r\n\t\tAND ISNULL(A.sub_acc_id,'') = ISNULL(B.sub_acc_id,'')\r\n\t\tAND A.td_curr_id = B.stl_aqn_curr_id\r\n\t\tAND B.stl_payout_hold = 'N'\r\n\t\tAND B.stl_payout_audit = 'Y'\r\n\t\tAND B.stl_gtw_id = @p_batch_gtw_id\r\n\t\tAND B.stl_status = 'ACTI'\r\n\t\tAND A.td_status = 'AWTD'\r\n\t\tAND B.reg_classification = @batch_merchant_type\r\n\t\t--AND A.order_stlmt_to = 'CCAvenue'\t-- Added on 20140804\r\n\t\tAND ( A.order_stlmt_to in('CCAvenue','CC-ICICI','CC-HDFC','CC-KMB') OR \r\n\t\t\t\t(A.order_stlmt_to = 'Merchant' AND A.order_comm_stlmt IN('Payable','Pre-Settled'))\r\n\t\t\t )\r\n\t\tAND A.td_batch_id IS NULL\r\n\t\tAND A.reg_id = @v_reg_id \r\n\t\tAND ISNULL(A.sub_acc_id,'') = ISNULL(@v_sub_acc_id,'')\r\n\t\t--AND EXISTS (SELECT 1 FROM CCAV_merchant_gateway_setting\r\n\t\t--\t\t\tWHERE reg_id = A.reg_id AND setting_payout_audit = 'Y'\r\n\t\t--\t\t\tAND setting_payout_hold = 'N'\r\n\t\t--\t\t\t)\r\n\t\tAND A.td_stl_curr_conv_rate IS NOT NULL\r\n\t\tAND A.td_type = CASE WHEN ISNULL(B.stl_include_debit, '') = 'Y' THEN A.td_type ELSE 'CR' END\r\n\t\tAND ISNULL(A.td_charge_TDR,'') <> ''\t\r\n\r\n    IF @@ROWCOUNT = 0\r\n    BEGIN\r\n\t   RAISERROR('Invalid request',16,1)\r\n    END\r\n/*\r\n\tCross check miscellaneous entries , If we want to add tax in misc amt for Credit/Debit\r\n*/\r\n\tINSERT INTO #temp_payout(reg_id,pay_to,td_type,pay_amt,bank_transfer,sub_acc_id) -- ,td_charge_TDR\r\n\tSELECT\tA.reg_id,B.stl_bank_beneficiary_name,A.td_type,\t\t\t\r\n\t\t\tSUM(\r\n\t\t\t\tCASE \r\n\t\t\t\tWHEN A.td_charge_TDR = 'Y' THEN\t\r\n\t\t\t\t\t((CASE WHEN A.td_trans_type IN('MISC','MSGCHRG') AND A.td_type = 'CR' THEN (td_amt+td_tax)\r\n\t\t\t\t\t\t\t WHEN A.td_trans_type IN('MISC','MSGCHRG') AND A.td_type = 'DR' THEN (td_amt+td_tax) ELSE td_amt END) * td_stl_curr_conv_rate-ISNULL(td_fee_perc,0)*td_stl_curr_conv_rate \r\n\t\t\t\t\t-ISNULL(td_fee_flat,0)*td_stl_curr_conv_rate - ISNULL(CASE WHEN A.td_trans_type IN('MISC','MSGCHRG') THEN  0.00 ELSE td_tax END,0)*td_stl_curr_conv_rate\r\n\t\t\t\t\t+ ISNULL(CASE WHEN A.td_type = 'CR' THEN A.td_tds WHEN A.td_type = 'DR' THEN A.td_tds ELSE 0 END,0)*td_stl_curr_conv_rate) -- TDS calculation \r\n\t\t\t\tWHEN A.td_charge_TDR = 'N' THEN\r\n\t\t\t\t\t((CASE WHEN A.td_trans_type IN('MISC','MSGCHRG') AND A.td_type = 'CR' THEN (td_amt+td_tax)\r\n\t\t\t\t\t\t WHEN A.td_trans_type IN('MISC','MSGCHRG') AND A.td_type = 'DR' THEN (td_amt+td_tax) ELSE td_amt END) * td_stl_curr_conv_rate-0.00 \r\n\t\t\t\t-0.00 - ISNULL(CASE WHEN A.td_trans_type IN('MISC','MSGCHRG') THEN  0.00 ELSE 0.00 END,0)*td_stl_curr_conv_rate)\r\n\t\t\t\tEND\r\n\t\t\t\t) \r\n\t\t\tAS pay_amt,\r\n\t\t\tCASE WHEN B.stl_bank_name IN('ICICI BANK LTD','ICICI BANK LIMITED') AND B.reg_classification = 'NDVP' THEN 'Y' \r\n\t\t\t\t WHEN B.stl_bank_name IN('STANDARD CHARTERED BANK') AND B.reg_classification = 'DVP' THEN 'Y' \r\n\t\t\t\t\tELSE 'N' END bank_transfer,\t\t\t\r\n\t\t\tISNULL(A.sub_acc_id,'') sub_acc_id--, A.td_charge_TDR\r\n\tFROM\tCCAV_payout_detail_temp A, CCAV_stl_bank_detail B\r\n\tWHERE\tA.td_date\t\t\t\t<= CONVERT(DATETIME,@batch_cutoff_date,105)\r\n\t\tAND A.td_curr_id\t\t\t= @batch_curr_id\r\n\t\tAND A.reg_id\t\t\t\t= B.reg_id \r\n\t\tAND A.td_curr_id\t\t\t= B.stl_aqn_curr_id \r\n\t\tAND ISNULL(A.sub_acc_id,'') = ISNULL(B.sub_acc_id,'')\r\n\t\tAND B.stl_payout_hold\t\t= 'N'\r\n\t\tAND B.stl_payout_audit\t\t= 'Y'\r\n\t\tAND B.stl_gtw_id\t\t\t= @p_batch_gtw_id\r\n\t\tAND B.stl_status\t\t\t= 'ACTI'\r\n\t\tAND A.td_status\t\t\t\t= 'ASSIGNED'\r\n\t\tAND A.td_batch_id\t\t\t= @batch_id \r\n\t\tAND A.reg_id\t\t\t\t= @v_reg_id \r\n\t\tAND ISNULL(A.sub_acc_id,'') = ISNULL(@v_sub_acc_id,'')\r\n\t\tAND B.reg_classification\t= @batch_merchant_type\t\r\n\t\tAND ISNULL(A.td_charge_TDR,'') <> ''\t\r\n\tGROUP BY A.reg_id,B.stl_bank_beneficiary_name,A.td_type, --A.td_charge_TDR,\r\n\t\t\tCASE WHEN B.stl_bank_name IN('ICICI BANK LTD','ICICI BANK LIMITED') AND B.reg_classification = 'NDVP' THEN 'Y' \r\n\t\t\tWHEN B.stl_bank_name IN('STANDARD CHARTERED BANK') AND B.reg_classification = 'DVP' THEN 'Y' \r\n\t\t\tELSE 'N' END,ISNULL(A.sub_acc_id,'')\r\n\r\n/*\r\n\tINSERT INTO #temp_payout(reg_id,pay_to,td_type,pay_amt,bank_transfer,sub_acc_id,td_charge_TDR)\r\n\tSELECT\tA.reg_id,B.stl_bank_beneficiary_name,A.td_type,\r\n\t\t\tSUM(\r\n\t\t\t(CASE WHEN A.td_trans_type IN('MISC','MSGCHRG') AND A.td_type = 'CR' THEN (td_amt+td_tax)\r\n\t\t\t\t\t WHEN A.td_trans_type IN('MISC','MSGCHRG') AND A.td_type = 'DR' THEN (td_amt+td_tax) ELSE td_amt END) * td_stl_curr_conv_rate-ISNULL(td_fee_perc,0)*td_stl_curr_conv_rate \r\n\t\t\t-ISNULL(td_fee_flat,0)*td_stl_curr_conv_rate - ISNULL(CASE WHEN A.td_trans_type IN('MISC','MSGCHRG') THEN  0.00 ELSE td_tax END,0)*td_stl_curr_conv_rate\r\n\t\t\t+ ISNULL(CASE WHEN A.td_type = 'CR' THEN A.td_tds WHEN A.td_type = 'DR' THEN A.td_tds ELSE 0 END,0)*td_stl_curr_conv_rate -- TDS calculation \r\n\t\t\t) pay_amt,\r\n\t\t\tCASE WHEN B.stl_bank_name IN('ICICI BANK LTD','ICICI BANK LIMITED') AND B.reg_classification = 'NDVP' THEN 'Y' \r\n\t\t\t\t WHEN B.stl_bank_name IN('STANDARD CHARTERED BANK') AND B.reg_classification = 'DVP' THEN 'Y' \r\n\t\t\t\t\tELSE 'N' END bank_transfer,\r\n\t\t\t\r\n\t\t\tISNULL(A.sub_acc_id,'') sub_acc_id,'Y' AS td_charge_TDR\r\n\tFROM\tCCAV_payout_detail_temp A, CCAV_stl_bank_detail B\r\n--\tWHERE\tA.td_date <= @v_batch_cutoff_date --CONVERT(DATETIME,@batch_cutoff_date,105)\r\n\tWHERE\tA.td_date <= CONVERT(DATETIME,@batch_cutoff_date,105)\r\n\t\tAND A.td_curr_id = @batch_curr_id\r\n\t\tAND A.reg_id = B.reg_id \r\n\t\tAND A.td_curr_id = B.stl_aqn_curr_id \r\n\t\tAND ISNULL(A.sub_acc_id,'') = ISNULL(B.sub_acc_id,'')\r\n\t\tAND B.stl_payout_hold = 'N'\r\n\t\tAND B.stl_payout_audit = 'Y'\r\n\t\tAND B.stl_gtw_id = @p_batch_gtw_id\r\n\t\tAND B.stl_status = 'ACTI'\r\n\t\tAND A.td_status = 'ASSIGNED'\r\n\t\tAND A.td_batch_id = @batch_id \r\n\t\tAND A.reg_id = @v_reg_id \r\n\t\tAND ISNULL(A.sub_acc_id,'') = ISNULL(@v_sub_acc_id,'')\r\n\t\tAND B.reg_classification = @batch_merchant_type\r\n\t\t--AND ISNULL(CASE WHEN A.reg_id IN(60701,61217,79568,79555,80698,80687,74949,79771,80098,81663,84381,85092,86022,88534,80185,87885,5841,92303,92296,93454,93450,93205,93182,\r\n\t\t--\t\t\t9714,9715,9716,9717,95034,68478,68470,100409,69358,101440,101661,101951,102042,9711,9712,9713,102127,9718,81724,97203) THEN 'N' ELSE A.td_setting_charge_tdr END,'Y') \r\n\t\t--\t\t\t= 'Y' -- ,104847\r\n\t\tAND EXISTS (SELECT 1 FROM CCAV_merchant_gateway_setting C\r\n\t\t\t\t\tWHERE C.reg_id = A.reg_id AND C.setting_charge_TDR = 'Y'\r\n\t\t\t\t\t)\r\n\tGROUP BY A.reg_id,B.stl_bank_beneficiary_name,A.td_type,td_charge_TDR,\r\n\t\t\t\t\tCASE WHEN B.stl_bank_name IN('ICICI BANK LTD','ICICI BANK LIMITED') AND B.reg_classification = 'NDVP' THEN 'Y' \r\n\t\t\t\t WHEN B.stl_bank_name IN('STANDARD CHARTERED BANK') AND B.reg_classification = 'DVP' THEN 'Y' \r\n\t\t\t\t\tELSE 'N' END,ISNULL(A.sub_acc_id,'')\r\n\r\n\r\n\tINSERT INTO #temp_payout(reg_id,pay_to,td_type,pay_amt,bank_transfer,sub_acc_id, td_charge_TDR)\r\n\tSELECT\tA.reg_id,B.stl_bank_beneficiary_name,A.td_type,\r\n\t\t\tSUM(\r\n\t\t\t(CASE WHEN A.td_trans_type IN('MISC','MSGCHRG') AND A.td_type = 'CR' THEN (td_amt+td_tax)\r\n\t\t\t\t\t WHEN A.td_trans_type IN('MISC','MSGCHRG') AND A.td_type = 'DR' THEN (td_amt+td_tax) ELSE td_amt END) * td_stl_curr_conv_rate-0.00 \r\n\t\t\t-0.00 - ISNULL(CASE WHEN A.td_trans_type IN('MISC','MSGCHRG') THEN  0.00 ELSE 0.00 END,0)*td_stl_curr_conv_rate\r\n\t\t\t) pay_amt,\r\n\t\t\tCASE WHEN B.stl_bank_name IN('ICICI BANK LTD','ICICI BANK LIMITED') AND B.reg_classification = 'NDVP' THEN 'Y' \r\n\t\t\t\t WHEN B.stl_bank_name IN('STANDARD CHARTERED BANK') AND B.reg_classification = 'DVP' THEN 'Y' \r\n\t\t\t\t\tELSE 'N' END bank_transfer,ISNULL(A.sub_acc_id,'') sub_acc_id, 'N' AS td_charge_TDR\r\n\tFROM\tCCAV_payout_detail_temp A, CCAV_stl_bank_detail B\r\n\tWHERE\tA.td_date <= CONVERT(DATETIME,@batch_cutoff_date,105)\r\n\t\tAND A.td_curr_id = @batch_curr_id\r\n\t\tAND A.reg_id = B.reg_id \r\n\t\tAND A.td_curr_id = B.stl_aqn_curr_id \r\n\t\tAND ISNULL(A.sub_acc_id,'') = ISNULL(B.sub_acc_id,'')\r\n\t\tAND B.stl_payout_hold = 'N'\r\n\t\tAND B.stl_payout_audit = 'Y'\r\n\t\tAND B.stl_gtw_id = @p_batch_gtw_id\r\n\t\tAND B.stl_status = 'ACTI'\r\n\t\tAND A.td_status = 'ASSIGNED'\r\n\t\tAND A.td_batch_id = @batch_id \r\n\t\tAND A.reg_id = @v_reg_id \r\n\t\tAND ISNULL(A.sub_acc_id,'') = ISNULL(@v_sub_acc_id,'')\r\n\t\tAND B.reg_classification = @batch_merchant_type\r\n\t\t--AND ISNULL(CASE WHEN A.reg_id IN(60701,61217,79568,79555,80698,80687,74949,79771,80098,81663,84381,85092,86022,88534,80185,87885,5841,92303,92296,93454,93450,93205,93182,\r\n\t\t--\t\t\t\t9714,9715,9716,9717,95034,68478,68470,100409,69358,101440,101661,101951,102042,9711,9712,9713,102127,9718,81724,97203) THEN 'N' ELSE A.td_setting_charge_tdr END,'Y') \r\n\t\t--\t\t\t\t= 'N' -- ,104847\r\n\t\tAND EXISTS (SELECT 1 FROM CCAV_merchant_gateway_setting C\r\n\t\t\t\t\tWHERE C.reg_id = A.reg_id AND C.setting_charge_TDR = 'N'\r\n\t\t\t\t\t)\r\n\tGROUP BY A.reg_id,B.stl_bank_beneficiary_name,A.td_type,td_charge_TDR,\r\n\t\tCASE WHEN B.stl_bank_name IN('ICICI BANK LTD','ICICI BANK LIMITED') AND B.reg_classification = 'NDVP' THEN 'Y' \r\n\t\t\t\t WHEN B.stl_bank_name IN('STANDARD CHARTERED BANK') AND B.reg_classification = 'DVP' THEN 'Y' \r\n\t\t\t\t\tELSE 'N' END,ISNULL(A.sub_acc_id,'')\r\n\r\n*/\r\n\t--SELECT * FROM #temp_payout\r\n/*\r\n\tIndividual payout entries is done here using group by \r\n*/\r\n\tINSERT INTO CCAV_payout_detail_temp(reg_id,td_date,td_type,td_curr_id,\r\n\t\t\t\t\ttd_stl_curr_id,\r\n\t\t\t\t\ttd_amt,td_remarks,\r\n\t\t\t\t\ttd_trans_type,td_batch_id,td_stl_via,sub_acc_id,td_charge_TDR)\r\n\tSELECT\treg_id,td_date,td_type,td_curr_id,td_stl_curr_id,td_amt,td_remarks,\r\n\t\t\t\t\ttd_trans_type,td_batch_id,\r\n\t\t\tCASE WHEN bank_transfer = 'Y' THEN 'BANK' WHEN td_amt >= 200000 THEN 'RTGS' \r\n\t\t\tWHEN td_amt < 200000 THEN 'NEFT' END td_stl_via,sub_acc_id,td_charge_TDR\r\n\tFROM\r\n\t(SELECT\treg_id,GETDATE()td_date,'DR' td_type,@batch_curr_id td_curr_id,'INR' AS td_stl_curr_id,--- ADDED ON 1NOV2013 REQUESTED BY HIMANSHU..\r\n\t\t\tSUM(CASE WHEN td_type = 'CR' THEN pay_amt ELSE 0 END) -\r\n\t\t\t\tSUM(CASE WHEN td_type = 'DR' THEN pay_amt ELSE 0 END) td_amt, -- NetPayable\r\n\t\t\t\t'Payout Batch Assigned' td_remarks,'Payout' td_trans_type ,@batch_id td_batch_id,\r\n\t\t\tbank_transfer,ISNULL(sub_acc_id,'') sub_acc_id,td_charge_TDR\r\n\tFrom\t#temp_payout\r\n\tWHERE\treg_id = @v_reg_id \r\n\t\tAND ISNULL(sub_acc_id,'') = ISNULL(@v_sub_acc_id,'')\r\n\tGROUP BY reg_id,bank_transfer,ISNULL(sub_acc_id,''), td_charge_TDR)b\r\n\t\r\n\tSET @td_payout_id = SCOPE_IDENTITY()\r\n\t\r\n\t--Assign td_payout_id to all the transactions considered in this payout\r\n\tUPDATE\tCCAV_payout_detail_temp\r\n\t\tSET td_payout_id = @td_payout_id\r\n\tWHERE\treg_id = @v_reg_id \r\n\t\tAND ISNULL(sub_acc_id,'') = ISNULL(@v_sub_acc_id,'')\r\n\t\tAND td_batch_id = @batch_id\r\n\t\tAND td_trans_type <> 'Payout'\r\n\t\tAND td_status = 'ASSIGNED'\r\n\t\tAND td_id <> @td_payout_id\r\n\t\r\n\tUPDATE\tB\r\n\t\tSET\tB.batch_total_amt = ISNULL(B.batch_total_amt,0) + ISNULL(A.batch_total_amt,0),\r\n\t\t\tB.batch_bank_transfer_amount = ISNULL(B.batch_bank_transfer_amount,0) + ISNULL(A.batch_bank_transfer_amount,0),\r\n\t\t\tB.batch_neft_amount = ISNULL(B.batch_neft_amount,0) + ISNULL(A.batch_neft_amount,0),\r\n\t\t\tB.batch_rtgs_amount = ISNULL(B.batch_rtgs_amount,0) + ISNULL(A.batch_rtgs_amount,0),\r\n\t\t\tB.batch_total_payouts = ISNULL(B.batch_total_payouts,0) + ISNULL(A.batch_total_payouts,0),\r\n\t\t\tB.batch_neft_count = ISNULL(B.batch_neft_count,0) + ISNULL(A.batch_neft_count,0),\r\n\t\t\tB.batch_rtgs_count = ISNULL(B.batch_rtgs_count,0) + ISNULL(A.batch_rtgs_count,0),\r\n\t\t\tB.batch_bank_transfer_count = ISNULL(B.batch_bank_transfer_count,0) + ISNULL(A.batch_bank_transfer_count,0)\r\n\tFROM\tCCAV_payout_batch B, ( SELECT td_batch_id,\r\n\t\t\t\tSUM(td_amt) batch_total_amt,\r\n\t\t\t\tSUM(CASE WHEN td_stl_via = 'BANK' THEN td_amt ELSE NULL END) batch_bank_transfer_amount,\r\n\t\t\t\tSUM(CASE WHEN td_stl_via = 'NEFT' THEN td_amt ELSE NULL END) batch_neft_amount,\r\n\t\t\t\tSUM(CASE WHEN td_stl_via = 'RTGS' THEN td_amt ELSE NULL END) batch_rtgs_amount,\r\n\t\t\t\tCOUNT(1) batch_total_payouts,\r\n\t\t\t\tCOUNT(CASE WHEN td_stl_via = 'NEFT' THEN td_amt ELSE NULL END) batch_neft_count,\r\n\t\t\t\tCOUNT(CASE WHEN td_stl_via = 'RTGS' THEN td_amt ELSE NULL END) batch_rtgs_count,\r\n\t\t\t\tCOUNT(CASE WHEN td_stl_via = 'BANK' THEN td_amt ELSE NULL END) batch_bank_transfer_count\r\n\t\t\tFROM CCAV_payout_detail_temp\r\n\t\t\t\tWHERE td_trans_type = 'Payout'\r\n\t\t\t\tAND td_batch_id = @batch_id\r\n\t\t\t\tAND td_id = @td_payout_id\r\n\t\t\tGROUP BY td_batch_id) A\r\n\tWHERE\tB.batch_id = @batch_id\r\n\t\tAND B.batch_id = A.td_batch_id \r\n\t\t\r\n\tSET @reg_id = NULL\r\n\tSET @td_payout_id = NULL\r\n\tTRUNCATE TABLE #temp_payout\r\n\r\nEND -- WHILE LOOP END\r\nCOMMIT TRAN Assigned_payout\r\nEND -- ASSIGNED IF END\r\n\r\nIF @activity = 'CLOSED'\r\nBEGIN\r\nBEGIN TRAN Closed_payout\r\n\r\n\tIF ISNULL(@v_rsl_id,'') <> ''--for one entry for reseller\r\n\tBEGIN\r\n\r\n\tCREATE TABLE #temp_rsl_payout(\r\n\tpay_to\tVARCHAR(255),\r\n\tpay_amt\tNUMERIC(13,3),\r\n\tbank_transfer CHAR(1)\r\n\t)\r\n\r\n\tINSERT INTO #temp_rsl_payout(pay_to,pay_amt,bank_transfer)\r\n\tSELECT\tB.stl_bank_beneficiary_name,\t\t\t\r\n\t\t\tSUM(A.td_amt) AS pay_amt,\r\n\t\t\tCASE WHEN B.stl_bank_name IN('ICICI BANK LTD','ICICI BANK LIMITED') AND B.rsl_classification = 'NDVP' THEN 'Y' \r\n\t\t\t\t WHEN B.stl_bank_name IN('STANDARD CHARTERED BANK') AND B.rsl_classification = 'DVP' THEN 'Y' \r\n\t\t\t\t\tELSE 'N' END bank_transfer\r\n\tFROM\tCCAV_payout_detail_temp A,ccav_registration C ,CCAV_reseller_setting B\r\n\tWHERE\t\r\n\t\tA.td_curr_id\t\t\t\t= @batch_curr_id\r\n\t\tAND A.td_curr_id\t\t\t= B.stl_aqn_curr_id \r\n\t\tAND A.reg_id\t\t\t\t= C.reg_id\r\n\t\tAND C.rsl_id\t\t\t\t= B.rsl_id\r\n\t\tAND B.stl_payout_hold\t\t= 'N'\r\n\t\tAND B.stl_payout_audit\t\t= 'Y'\r\n\t\tAND A.td_batch_id\t\t\t= @batch_id\r\n\t\tAND A.td_trans_type\t\t\t= 'Payout'\r\n\t\tAND B.rsl_classification\t= @batch_merchant_type\t\r\n\tGROUP BY B.stl_bank_beneficiary_name,\r\n\t\t\tCASE WHEN B.stl_bank_name IN('ICICI BANK LTD','ICICI BANK LIMITED') AND B.rsl_classification = 'NDVP' THEN 'Y' \r\n\t\t\tWHEN B.stl_bank_name IN('STANDARD CHARTERED BANK') AND B.rsl_classification = 'DVP' THEN 'Y' \r\n\t\t\tELSE 'N' END\r\n\r\n\tINSERT INTO CCAV_payout_detail_temp(td_date,td_type,td_curr_id,\r\n\t\t\t\t\ttd_stl_curr_id,\r\n\t\t\t\t\ttd_amt,td_remarks,\r\n\t\t\t\t\ttd_trans_type,td_batch_id,td_stl_via,rsl_id)\r\n\tSELECT\ttd_date,td_type,td_curr_id,td_stl_curr_id,td_amt,td_remarks,\r\n\t\t\t\t\ttd_trans_type,td_batch_id,\r\n\t\t\tCASE WHEN bank_transfer = 'Y' THEN 'BANK' WHEN td_amt >= 200000 THEN 'RTGS' \r\n\t\t\tWHEN td_amt < 200000 THEN 'NEFT' END td_stl_via,rsl_id\r\n\tFROM\r\n\t(SELECT\tGETDATE()td_date,'DR' td_type,@batch_curr_id td_curr_id,'INR' AS td_stl_curr_id,\r\n\t\t\tSUM(pay_amt) td_amt, -- NetPayable\r\n\t\t\t\t'Payout Batch Assigned' td_remarks,'Payout' td_trans_type ,@batch_id td_batch_id,\r\n\t\t\tbank_transfer,@v_rsl_id as rsl_id\r\n\tFrom\t#temp_rsl_payout\r\n\tGROUP BY bank_transfer)b\r\n\t\r\n\tSET @td_payout_id = SCOPE_IDENTITY()\r\n\t\r\n\t--Assign td_payout_id to all the reg_ids whose payout is completed\r\n\tUPDATE\tCCAV_payout_detail_temp\r\n\t\tSET td_payout_id = @td_payout_id\r\n\tWHERE\ttd_batch_id = @batch_id\r\n\t\tAND td_trans_type = 'Payout'\r\n\t\tAND td_id <> @td_payout_id\r\n\r\n\tEND\r\n\r\n\tUPDATE\tCCAV_payout_detail_temp\r\n\t\tSET\ttd_status = 'CLOSED'\r\n\tWHERE\ttd_batch_id = @batch_id\r\n\r\n\tINSERT INTO CCAV_payout_detail(\r\n\t\t\ttd_id,tracking_id,td_misc_id,td_date,td_settle_date,td_type,td_curr_id,\r\n\t\t\ttd_amt,td_bank_fee_perc,td_bank_fee_flat,td_fee_perc,td_fee_flat,td_tax,td_tds,order_stlmt_to,order_comm_stlmt,\r\n\t\t\ttd_stl_curr_id,td_stl_curr_conv_rate,\r\n\t\t\ttd_remarks,td_payout_by,td_trans_type,td_payout_id,td_batch_id,td_status,reg_id,td_stl_via,td_setting_charge_tdr,\r\n\t\t\tsub_acc_id, td_split_payout,td_charge_TDR\r\n\t\t\t\r\n\t\t\t)\r\n\tSELECT\ttd_id,tracking_id,td_misc_id,td_date,td_settle_date,td_type,td_curr_id,\r\n\t\t\ttd_amt,td_bank_fee_perc,td_bank_fee_flat,td_fee_perc,td_fee_flat,td_tax,td_tds,order_stlmt_to,order_comm_stlmt,\r\n\t\t\ttd_stl_curr_id,td_stl_curr_conv_rate,\r\n\t\t\ttd_remarks,td_payout_by,td_trans_type,td_payout_id,td_batch_id,td_status,reg_id,td_stl_via,td_setting_charge_tdr,\r\n\t\t\tsub_acc_id, td_split_payout,td_charge_TDR\r\n\tFROM\tCCAV_payout_detail_temp \r\n\tWHERE\ttd_batch_id = @batch_id\r\n\t\tAND td_status = 'CLOSED'\r\n\t\t\r\n\tUPDATE\tCCAV_payout_batch \r\n\t\tSET batch_status = 'CLOSED'\r\n\tWHERE batch_id = @batch_id\r\nCOMMIT TRAN Closed_payout\r\nEND\r\n\r\nEND TRY\r\nBEGIN CATCH\r\nIF @@TRANCOUNT <> 0\r\n\tROLLBACK WORK\r\n\tINSERT Exception_log (UserName,procedurename,errorNumber, errorSeverity, errorState, errorline, errorMessage,event_occured) \t\r\n\tVALUES (SUSER_SNAME(), ERROR_PROCEDURE(), ERROR_NUMBER(), ERROR_SEVERITY(), ERROR_LINE(), ERROR_STATE(), ERROR_MESSAGE(), GETDATE())  \r\n\r\n\tSET @p_iserror = 'Y'\r\n\tSET @p_error_desc = ERROR_MESSAGE()\r\n\r\nEND CATCH\r\nEND"

In [3]:
b = print(a)

-- @ Title	: ccav$assign_payouts
-- @ Version : 1.0
-- @ Author : Rakesh
-- @ Company : Avenues (I) Pvt. Limited.
-- @ Description : Assigning Payouts
-- @ Created : 12-Dec-2011
-- @ Modified : 15-SEP-2016 Supriya Ghude - Reg_id - 104847 removed from the list
-- @ Modified : 17-SEP-2018 Supriya Ghude - Added code for - [td_charge_TDR]
-- @ Modified : 02-OCT-2018 Supriya Ghude - Added code for - [stl_include_debit]
-- @ Modified : 06-SEP-2018 Kalyani Pandey - Added td_trans_type = 'MSGCHRG' with 'MISC'
-- @ Modified : 24-April-2019 Dinesh - Changed condition for @batch_cutoff_date date format in WHERE clause.
-- @ Modified : 25-APR-2019 Supriya Ghude - Code changes for - td_charge_TDR
-- @ Modified : 17-MAY-2019 Kalyani Pandey - Added 'CC-ICICI','CC-HDFC','CC-KMB' nodal with 'CCAvenue' in order_stlmt_to
-- @ Modified : 22-AUG-2019 Kalyani Pandey - Added code for checking reseller with setting_rsl_consolidated_stlmt = 'Y' for payout
--											- Added code for consolidate payout resell

In [4]:
b

In [7]:
"a\nb"

'a\nb'

In [8]:
print("a\nb")

a
b


In [12]:
"a\nb".replace("\n", "    ")

'a    b'

In [26]:
a = a.replace("\t", "    ")

In [27]:
a = a.replace("\r\n", "<br>")

In [28]:
a

"-- @ Title    : ccav$assign_payouts<br>-- @ Version : 1.0<br>-- @ Author : Rakesh<br>-- @ Company : Avenues (I) Pvt. Limited.<br>-- @ Description : Assigning Payouts<br>-- @ Created : 12-Dec-2011<br>-- @ Modified : 15-SEP-2016 Supriya Ghude - Reg_id - 104847 removed from the list<br>-- @ Modified : 17-SEP-2018 Supriya Ghude - Added code for - [td_charge_TDR]<br>-- @ Modified : 02-OCT-2018 Supriya Ghude - Added code for - [stl_include_debit]<br>-- @ Modified : 06-SEP-2018 Kalyani Pandey - Added td_trans_type = 'MSGCHRG' with 'MISC'<br>-- @ Modified : 24-April-2019 Dinesh - Changed condition for @batch_cutoff_date date format in WHERE clause.<br>-- @ Modified : 25-APR-2019 Supriya Ghude - Code changes for - td_charge_TDR<br>-- @ Modified : 17-MAY-2019 Kalyani Pandey - Added 'CC-ICICI','CC-HDFC','CC-KMB' nodal with 'CCAvenue' in order_stlmt_to<br>-- @ Modified : 22-AUG-2019 Kalyani Pandey - Added code for checking reseller with setting_rsl_consolidated_stlmt = 'Y' for payout<br>--       

In [23]:
print(a)

-- @ Title    : ccav$assign_payouts
-- @ Version : 1.0
-- @ Author : Rakesh
-- @ Company : Avenues (I) Pvt. Limited.
-- @ Description : Assigning Payouts
-- @ Created : 12-Dec-2011
-- @ Modified : 15-SEP-2016 Supriya Ghude - Reg_id - 104847 removed from the list
-- @ Modified : 17-SEP-2018 Supriya Ghude - Added code for - [td_charge_TDR]
-- @ Modified : 02-OCT-2018 Supriya Ghude - Added code for - [stl_include_debit]
-- @ Modified : 06-SEP-2018 Kalyani Pandey - Added td_trans_type = 'MSGCHRG' with 'MISC'
-- @ Modified : 24-April-2019 Dinesh - Changed condition for @batch_cutoff_date date format in WHERE clause.
-- @ Modified : 25-APR-2019 Supriya Ghude - Code changes for - td_charge_TDR
-- @ Modified : 17-MAY-2019 Kalyani Pandey - Added 'CC-ICICI','CC-HDFC','CC-KMB' nodal with 'CCAvenue' in order_stlmt_to
-- @ Modified : 22-AUG-2019 Kalyani Pandey - Added code for checking reseller with setting_rsl_consolidated_stlmt = 'Y' for payout
--                                            - Adde